#**Importações**

**Instalações de bibliotecas não nativas**

In [ ]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 198 kB 66.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=2c3854defc0b0a375584a9b3d871deea376c443a59f496b66c32b8594ae7ac15
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
pip install fsspec

     |████████████████████████████████| 132 kB 12.6 MB/s 


In [ ]:
pip install gcsfs

     |████████████████████████████████| 1.1 MB 14.6 MB/s 
     |████████████████████████████████| 271 kB 66.3 MB/s 
     |████████████████████████████████| 160 kB 50.7 MB/s 
     |████████████████████████████████| 192 kB 82.4 MB/s 


**BIBLIOTECAS**

In [ ]:
import os
from google.cloud import storage

from pyspark.sql import SparkSession, Window
spark = SparkSession.builder.appName('GCSFilesRead').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.functions import to_timestamp, to_date

from pyspark.sql.types import *
import pandas as pd

**Criando esquema e importando dataframe**

In [ ]:
customSchema = StructType([
   StructField("id_video", StringType(),True),
   StructField("titulo_video", StringType(),True),
   StructField("publicado_em", StringType(),True),
   StructField("id_canal", StringType(),True),
   StructField("nome_canal", StringType(),True),
   StructField("data_destaque", StringType(),True),
   StructField("cont_visualizacao", IntegerType(),True),
   StructField("curtidas", IntegerType(),True),
   StructField("nao_curtidas", IntegerType(),True),
   StructField("cont_comentarios", IntegerType(),True),
   StructField("comentarios_desabilitados", StringType(),True),
   StructField("curtidas_desabilitadas", StringType(),True),
   StructField("pais", StringType(),True),
   StructField("categoria", StringType(),True)

])

schema = customSchema

In [ ]:
dfspark = pd.read_csv('gs://projetofinalgrupo8/saida/youtube_tratado_pandas.csv', sep=',',encoding='UTF-8', header=0)
df= spark.createDataFrame(dfspark,schema=schema)

In [ ]:
df.show(5)

+-----------+--------------------+-------------------+--------------------+--------------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+
|   id_video|        titulo_video|       publicado_em|            id_canal|          nome_canal|      data_destaque|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|
+-----------+--------------------+-------------------+--------------------+--------------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|2020-08-11 22:21:49|UCGfBwrCoi9ZJjKiU...|       Pietro Guedes|2020-08-12 00:00:00|           263835|   85095|         487|            4500|                    false|                 false|  BR|pessoas_e_blogs|
|jbGRowa5tIk|ITZY “Not S

In [ ]:
df.printSchema()

root
 |-- id_video: string (nullable = true)
 |-- titulo_video: string (nullable = true)
 |-- publicado_em: string (nullable = true)
 |-- id_canal: string (nullable = true)
 |-- nome_canal: string (nullable = true)
 |-- data_destaque: string (nullable = true)
 |-- cont_visualizacao: integer (nullable = true)
 |-- curtidas: integer (nullable = true)
 |-- nao_curtidas: integer (nullable = true)
 |-- cont_comentarios: integer (nullable = true)
 |-- comentarios_desabilitados: string (nullable = true)
 |-- curtidas_desabilitadas: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- categoria: string (nullable = true)



#**Criação e remoção de colunas**

**Criação colunas publicação de data e data de destaque**

In [ ]:
#CRIAR COLUNAS DE PUBLICAÇÃO DE DATA E DATA DE DESTAQUE
df = df.withColumn("publicado_em_data", F.lit(F.substring('publicado_em', 1, 10))).withColumn("data_destaque1", F.lit(F.substring('data_destaque', 1, 10)))
df.show(1)

+-----------+--------------------+-------------------+--------------------+-------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|   id_video|        titulo_video|       publicado_em|            id_canal|   nome_canal|      data_destaque|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque1|
+-----------+--------------------+-------------------+--------------------+-------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|2020-08-11 22:21:49|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|2020-08-12 00:00:00|           263835|   85095|         487|            4500|                    fals

**Remoção colunas publicado_em e data_destaque**

In [ ]:
#DROPAR COLUNA PUBLICADO_EM
df = df.drop('publicado_em')
df.show(1)

+-----------+--------------------+--------------------+-------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|      data_destaque|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque1|
+-----------+--------------------+--------------------+-------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|2020-08-12 00:00:00|           263835|   85095|         487|            4500|                    false|                 false|  BR|pessoas_e_blogs|       2020-08-11|    2020-08-12|


In [ ]:
#DROPAR COLUNA DE DESTAQUE
df = df.drop('data_destaque')
df.show(1)

+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque1|
+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|           263835|   85095|         487|            4500|                    false|                 false|  BR|pessoas_e_blogs|       2020-08-11|    2020-08-12|
+-----------+--------------------+--------------------+-------------+-----------

**Renomeando coluna data destaque**

In [ ]:
df = df.withColumnRenamed("data_destaque1", "data_destaque")
df.show(1)

+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque|
+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|           263835|   85095|         487|            4500|                    false|                 false|  BR|pessoas_e_blogs|       2020-08-11|   2020-08-12|
+-----------+--------------------+--------------------+-------------+---------------

#**Criação de Insights**

**Relação curtida por visualização em porcentagem**



In [ ]:
df = (df.withColumn("like_por_visualizacao_100", F.round((F.col("curtidas")/ F.col("cont_visualizacao")*100),2))) 
df.show(1)

+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|
+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|           263835|   85095|         487|            4500|                    false|                 false|  BR|pessoas_e_blogs|       2020-08-11|   2020-08-12|       

**Relação descurtida por visualização em porcentagem**


In [ ]:
df = (df.withColumn("deslike_por_visualizacao_100", F.round((F.col("nao_curtidas")/ F.col("cont_visualizacao")*100),2))) 

df.show(1)

+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|deslike_por_visualizacao_100|
+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|           263835|   85095|         487|            4500|                    fa

**Relação de comentários por visualização em porcentagem**


In [ ]:
df = (df.withColumn("coment_por_visualizacao_100", F.round((F.col("cont_comentarios")/ F.col("cont_visualizacao")*100),2))) 
df.show(1)

+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|deslike_por_visualizacao_100|coment_por_visualizacao_100|
+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Gu

#**Transformando os tipos das colunas para data**



In [ ]:
df = df.withColumn("publicado_em_data", F.col("publicado_em_data").cast("date"))
df = df.withColumn("data_destaque", F.col("data_destaque").cast("date"))

df.select(F.col("publicado_em_data"), F.col("data_destaque")).show(5)
df.printSchema()

+-----------------+-------------+
|publicado_em_data|data_destaque|
+-----------------+-------------+
|       2020-08-11|   2020-08-12|
|       2020-08-11|   2020-08-12|
|       2020-08-10|   2020-08-12|
|       2020-08-11|   2020-08-12|
|       2020-08-11|   2020-08-12|
+-----------------+-------------+
only showing top 5 rows

root
 |-- id_video: string (nullable = true)
 |-- titulo_video: string (nullable = true)
 |-- id_canal: string (nullable = true)
 |-- nome_canal: string (nullable = true)
 |-- cont_visualizacao: integer (nullable = true)
 |-- curtidas: integer (nullable = true)
 |-- nao_curtidas: integer (nullable = true)
 |-- cont_comentarios: integer (nullable = true)
 |-- comentarios_desabilitados: string (nullable = true)
 |-- curtidas_desabilitadas: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- publicado_em_data: date (nullable = true)
 |-- data_destaque: date (nullable = true)
 |-- like_por_visualizacao_100: dou

#**Rank de videos dentro de um canal**


In [ ]:
#RANK DOS VÍDEOS COM MAIS CURTIDA/VISUALIZAÇÃO EM UM CANAL
w0 = Window.partitionBy(F.col("nome_canal")).orderBy("like_por_visualizacao_100")

(df.withColumn("RANK",F.rank().over(w0))
  .withColumn("curtida_por_visualizacao",F.max(F.col("like_por_visualizacao_100")).over(w0))
  .select("RANK","curtida_por_visualizacao",'titulo_video','nome_canal',"pais","publicado_em_data").show(15)
)

+----+------------------------+--------------------+--------------------+----+-----------------+
|RANK|curtida_por_visualizacao|        titulo_video|          nome_canal|pais|publicado_em_data|
+----+------------------------+--------------------+--------------------+----+-----------------+
|   1|                   10.17|#Shorts ¿Es esta ...| PONGAMOSLO A PRU...|  MX|       2021-10-07|
|   2|                   10.18|#Shorts ¿Es esta ...| PONGAMOSLO A PRU...|  MX|       2021-10-07|
|   3|                    10.2|#Shorts ¿Es esta ...| PONGAMOSLO A PRU...|  MX|       2021-10-07|
|   4|                   10.21|#Shorts ¿El mejor...| PONGAMOSLO A PRU...|  MX|       2021-11-09|
|   5|                   10.22|#Shorts ¿El mejor...| PONGAMOSLO A PRU...|  MX|       2021-11-09|
|   6|                   10.24|#Shorts ¿El mejor...| PONGAMOSLO A PRU...|  MX|       2021-11-09|
|   7|                   10.25|#Shorts ¿Es esta ...| PONGAMOSLO A PRU...|  MX|       2021-10-07|
|   8|                   10.29

In [ ]:
#RANK DOS VÍDEOS COM MAIS DESCURTIDA/VISUALIZAÇÃO EM UM CANAL
w0 = Window.partitionBy(F.col("nome_canal")).orderBy("deslike_por_visualizacao_100")

(df.withColumn("RANK",F.row_number().over(w0))
  .withColumn("descurtida_por_visualizacao",F.max(F.col("deslike_por_visualizacao_100")).over(w0))
  .select("RANK","descurtida_por_visualizacao",'titulo_video','nome_canal',"pais","publicado_em_data").show(15)
)

+----+---------------------------+--------------------+--------------------+----+-----------------+
|RANK|descurtida_por_visualizacao|        titulo_video|          nome_canal|pais|publicado_em_data|
+----+---------------------------+--------------------+--------------------+----+-----------------+
|   1|                       0.11|#Shorts, puse a p...| PONGAMOSLO A PRU...|  MX|       2021-08-09|
|   2|                       0.11|#Shorts, puse a p...| PONGAMOSLO A PRU...|  MX|       2021-08-09|
|   3|                       0.11|#Shorts Puse a pr...| PONGAMOSLO A PRU...|  MX|       2021-08-12|
|   4|                       0.11|#Shorts, puse a p...| PONGAMOSLO A PRU...|  MX|       2021-08-09|
|   5|                       0.11|#Shorts Puse a pr...| PONGAMOSLO A PRU...|  MX|       2021-08-12|
|   6|                       0.11|#Shorts, puse a p...| PONGAMOSLO A PRU...|  MX|       2021-08-09|
|   7|                       0.11|#Shorts Puse a pr...| PONGAMOSLO A PRU...|  MX|       2021-08-12|


#**Destaques top 10**

**Videos com mais visualizações**

In [ ]:
(df.groupBy(F.col("titulo_video")).agg(
    F.max("cont_visualizacao").alias("total_visualizacao"),
    F.max("publicado_em_data").alias("publicado_em_data"),
    F.max("data_destaque").alias("data_destaque")).orderBy('total_visualizacao', ascending=False).show(10)
)

+-------------------------+------------------+-----------------+-------------+
|             titulo_video|total_visualizacao|publicado_em_data|data_destaque|
+-------------------------+------------------+-----------------+-------------+
|BTS (방탄소년단) 'Butt...|         296314174|       2021-05-21|   2021-06-04|
|BTS (방탄소년단) 'Dyna...|         262319276|       2020-08-21|   2020-08-31|
|     Turn into orbeez ...|         206202284|       2021-07-03|   2021-08-08|
|     Filhaal2 Mohabbat...|         202091414|       2021-07-06|   2021-07-16|
|BTS (방탄소년단) 'Perm...|         194795844|       2021-07-09|   2021-07-24|
|     JETSKI WAX PRANK!...|         194625542|       2021-07-04|   2021-07-15|
|      LISA - 'LALISA' M/V|         192376395|       2021-09-10|   2021-09-22|
|     BLACKPINK - 'Ice ...|         184778248|       2020-08-28|   2020-09-05|
|     INSANE strawberry...|         164362471|       2021-05-08|   2021-05-19|
|BTS (방탄소년단) 'Life...|         161912058|       2020-11-20|   2020-

**Canais com maior soma de visualização**

In [ ]:
(df.groupBy(F.col("nome_canal")).agg(
    F.sum("cont_visualizacao").alias("soma_visualizacao"),
    F.round(F.avg("cont_visualizacao"),2).alias("media_visualizacao"),
    F.sum("curtidas").alias("soma_curtidas"),
    F.round(F.avg("curtidas"),2).alias("media_curtidas"),
    F.sum("nao_curtidas").alias("soma_naocurtidas"),
    F.max("publicado_em_data").alias("publicado_em_data"),
    F.max("data_destaque").alias("data_destaque"),
    F.round(F.avg("nao_curtidas"),2).alias("media_naocurtidas")).orderBy('soma_visualizacao', ascending=False).show(10)
)

+-----------------+-----------------+------------------+-------------+--------------+----------------+-----------------+-------------+-----------------+
|       nome_canal|soma_visualizacao|media_visualizacao|soma_curtidas|media_curtidas|soma_naocurtidas|publicado_em_data|data_destaque|media_naocurtidas|
+-----------------+-----------------+------------------+-------------+--------------+----------------+-----------------+-------------+-----------------+
|        BLACKPINK|      58230237271|      2.82396883E7|   5596684546|    2714202.01|       117843253|       2021-10-28|   2021-11-01|         57149.98|
|      HYBE LABELS|      47383301434|     3.197253808E7|   4954981634|     3343442.4|        30903181|       2021-10-27|   2021-11-04|         20852.35|
|           SMTOWN|      46008908926|     2.024149095E7|   2845074443|    1251682.55|        40529445|       2021-10-25|   2021-11-06|         17830.82|
|JYP Entertainment|      38480556253|     1.798156834E7|   2296325821|    1073049.

**Videos com mais de 50% de curtidas em relação as visualizações**

In [ ]:
df.where((F.col("pais") == 'BR')).filter("like_por_visualizacao_100 > 50").show(20)

+-----------+--------------------+--------------------+----------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|   id_video|        titulo_video|            id_canal|nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|deslike_por_visualizacao_100|coment_por_visualizacao_100|
+-----------+--------------------+--------------------+----------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|CuyTC8FLICY|Anitta - Girl Fro...|UCqjjyPUghDSSKFBA...|    Anitta|           544091|  32564

#SALVANDO NO BUCKET

**Salvando na raiz do notebook, pois não foi possivel salvar o dataframe do pyspark na GCP**

In [ ]:
df.write.format("com.databricks.spark.csv").option("header", "true")\
                                           .option("inferschema", "true")\
                                           .option("delimiter", ",")\
                                           .save("/content/data_tratado_pyspark")

**Devido ao tamanho do arquivo ele se reparte em dois no salvamento, assim é necessário carrega-lo novamente para poder salvar através do pandas na GCP**

In [ ]:
#COPIAR OS ENDEREÇOS DOS ARQUIVOS NA PASTA "data_tratado_pyspark"
df_pandas1 = pd.read_csv('/content/data_tratado_pyspark/part-00000-ca6ec06c-1839-48be-9def-887040fde7fb-c000.csv')
df_pandas2 = pd.read_csv('/content/data_tratado_pyspark/part-00001-ca6ec06c-1839-48be-9def-887040fde7fb-c000.csv')
df_pandas2.head(1)

,id_video,titulo_video,id_canal,nome_canal,cont_visualizacao,curtidas,nao_curtidas,cont_comentarios,comentarios_desabilitados,curtidas_desabilitadas,pais,categoria,publicado_em_data,data_destaque,like_por_visualizacao_100,deslike_por_visualizacao_100,coment_por_visualizacao_100
0,qDNssUZPZmw,Teri Aadat - Behind The Scenes ft. Siddharth N...,UCoFxUkg6cLuhlY3Ll8SHTDQ,Anushka Sen,254908,23129,629,1443,False,False,IN,entretenimento,2021-03-26,2021-03-28,9.07,0.25,0.57


**Verificação do tamanho dos datasets que foram repartidos**

In [ ]:
print(df_pandas1.shape)
print(df_pandas2.shape)

(510976, 17)
(510645, 17)


**Concatenação e verificação do tamanho**

In [ ]:
df_pandas_def = pd.concat([df_pandas1, df_pandas2])
df_pandas_def.shape

(1021621, 17)

**Salvando no Bucket Final**

In [ ]:
serviceAccount = '/content/projetofinalgrupo8-2dcd866c3f46.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

client = storage.Client()
bucket = client.get_bucket('projetofinalgrupo8')
bucket.blob('saida/youtube_tratado_pyspark.csv').upload_from_string(df_pandas_def.to_csv(index=False), 'text/csv')